In [1]:
!pip install -q tensorflow==2.1

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 3.8MB 52.9MB/s 
     |████████████████████████████████| 450kB 67.6MB/s 
     |████████████████████████████████| 81kB 13.7MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.


In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 33928, done.
remote: Total 33928 (delta 0), reused 0 (delta 0), pack-reused 33928
Receiving objects: 100% (33928/33928), 512.04 MiB | 14.87 MiB/s, done.
Resolving deltas: 100% (21729/21729), done.
Checking out files: 100% (3173/3173), done.


In [0]:
#import os
#os.environ['PYTHONPATH'] += ":/content/models" # ":/usr/local/lib/python3.6/dist-packages"
#!export PYTHONPATH=$PYTHONPATH:models

In [0]:
# install requirements
import sys
sys.path.append('models')

In [3]:
!pip install --user -q -r models/official/requirements.txt
# you may have to restart the runtime

     |████████████████████████████████| 92kB 5.3MB/s 
     |████████████████████████████████| 102kB 5.9MB/s 
     |████████████████████████████████| 1.0MB 55.9MB/s 
     |████████████████████████████████| 21.8MB 141kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
tf.__version__

'2.1.0'

In [0]:
# import as test
from official.nlp.bert import tokenization
from official.nlp.bert import run_classifier

In [5]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [0]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
import tensorflow_datasets as tfds
data, info = tfds.load("imdb_reviews/plain_text", with_info=True)
train_data, test_data = data['train'], data['test']

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


In [0]:
#train_data = train_data.shard(num_shards=4, index=0)#.batch(32)#.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
for t in train_data.take(1):
  print(t)

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is

In [0]:
from official.nlp.bert import classifier_data_lib
from official.nlp.bert import tokenization
import tensorflow_hub as hub

label_list = [0,1]
max_seq_length = 128
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1",
                            trainable=True)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

def to_listed_line(line):
  #tf.print(line['label'])
  return (line['text'], line['label'])

def to_feature(text, label):
  example = classifier_data_lib.InputExample(guid = None,
                                            text_a = text.numpy(), 
                                            text_b = None, 
                                            label = label.numpy())
  feature = classifier_data_lib.convert_single_example(5, example, label_list,
                                     max_seq_length, tokenizer)
  
  return (feature.input_ids, feature.input_mask, 
          feature.segment_ids, feature.label_id)
  # feature = InputFeatures(input_ids, input_mask, segment_ids, label_id)

def convert_to_feature(text, label):
  #return tf.py_function(to_feature, inp=[text, label], 
  #                               Tout=[tf.int64, tf.int64, tf.int64, tf.int64])
  x_ids, x_mask, x_segment, y = tf.py_function(to_feature, inp=[text, label], 
                                 Tout=[tf.int64, tf.int64, tf.int64, tf.int64])
  tf.reshape(x_ids, [128]) 
  #return ((x_ids, x_mask, x_segment), y)
  
  # set_shape or padded_batch() ?
  x_ids.set_shape([128])
  x_mask.set_shape([128])
  x_segment.set_shape([128])
  y.set_shape([])

  x = {
        #'text': line['text'],
        'input_word_ids': x_ids,
        'input_mask': x_mask,
        'input_type_ids': x_segment
    }
  #y = y #line['label']
  return (x, y)
  #

In [11]:
for t in train_data.take(1): #.as_numpy_iterator():
  print(t)

{'label': <tf.Tensor: shape=(), dtype=int64, numpy=1>, 'text': <tf.Tensor: shape=(), dtype=string, numpy=b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is

In [0]:
train_data, test_data = data['train'], data['test']
# train
train_data = train_data.map(
      to_listed_line,
      num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_data = train_data.map(
      convert_to_feature,
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
# test
test_data = test_data.map(
      to_listed_line,
      num_parallel_calls=tf.data.experimental.AUTOTUNE)

test_data = test_data.map(
      convert_to_feature,
      num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [10]:
train_data.element_spec

({'input_mask': TensorSpec(shape=(128,), dtype=tf.int64, name=None),
  'input_type_ids': TensorSpec(shape=(128,), dtype=tf.int64, name=None),
  'input_word_ids': TensorSpec(shape=(128,), dtype=tf.int64, name=None)},
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [0]:
train_data = train_data.shuffle(100)
train_data = train_data.repeat()
"""
padded_shapes = ({'input_mask': [128],
                  'input_type_ids': [128],
                  'input_word_ids': [128]},
                  [])
train_data = train_data.padded_batch(32, padded_shapes=padded_shapes, drop_remainder=True)
"""
train_data = train_data.batch(32, drop_remainder=True)
train_data = train_data.prefetch(1024)

test_data = test_data.batch(32, drop_remainder=True)
test_data = test_data.prefetch(1024)

In [11]:
train_data.element_spec

({'input_mask': TensorSpec(shape=(32, 128), dtype=tf.int64, name=None),
  'input_type_ids': TensorSpec(shape=(32, 128), dtype=tf.int64, name=None),
  'input_word_ids': TensorSpec(shape=(32, 128), dtype=tf.int64, name=None)},
 TensorSpec(shape=(32,), dtype=tf.int64, name=None))

In [0]:
for t in train_data.take(1):#.as_numpy_iterator():
  print(t)

({'input_word_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[  101,   100,  8945, ...,  8663,  9064,   102],
       [  101,   100,  8815, ...,   119,   100,   102],
       [  101,   100,  8541, ..., 10238, 11765,   102],
       ...,
       [  101,   100, 11099, ...,   112,   161,   102],
       [  101,   100,  8310, ...,   135,   133,   102],
       [  101,   100,   162, ...,  9947,  8968,   102]])>, 'input_mask': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])>, 'input_type_ids': <tf.Tensor: shape=(32, 128), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(32,), dtype=int6

In [11]:
tf.keras.backend.clear_session()

# max_seq_length = 128  # done earlier.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
input_type_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="input_type_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_zh_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, input_type_ids])

output = tf.keras.layers.Dense(2, activation='softmax', name="output")(pooled_output)
#drop = tf.keras.layers.Dropout(0.5)(pooled_output)

model = tf.keras.Model(
      inputs={
          'input_word_ids': input_word_ids,
          'input_mask': input_mask,
          'input_type_ids': input_type_ids
      },
      outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 102267649   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [0]:
from official.nlp import optimization

train_data_size = 25000
epochs = 5
train_batch_size = 32
initial_lr = 2e-5
steps_per_epoch = int(train_data_size / train_batch_size)
warmup_steps = int(epochs * train_data_size * 0.1 / train_batch_size)

optimizer = optimization.create_optimizer(
        initial_lr, steps_per_epoch * epochs, warmup_steps)

def metric_fn():
  return tf.keras.metrics.SparseCategoricalAccuracy(
      'test_accuracy', dtype=tf.float32)

def get_loss_fn(num_classes, loss_factor=1.0):
  """Gets the classification loss function."""

  def classification_loss_fn(labels, logits):
    """Classification loss."""
    labels = tf.squeeze(labels)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    one_hot_labels = tf.one_hot(
        tf.cast(labels, dtype=tf.int32), depth=num_classes, dtype=tf.float32)
    per_example_loss = -tf.reduce_sum(
        tf.cast(one_hot_labels, dtype=tf.float32) * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    loss *= loss_factor
    return loss
  return classification_loss_fn

loss_multiplier = 1.0
loss_fn = get_loss_fn(2, loss_factor=loss_multiplier)

#model.compile(optimizer=optimizer, loss=loss_fn, metrics=[metric_fn()])
model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# WHAT WAS WRONG? LOSS OR OPTIMIZER? optimizer: adam doesn't work for default initial learning rate

In [0]:
train_data_size = 25000
epochs = 5
train_batch_size = 32
initial_lr = 2e-5
steps_per_epoch = int(train_data_size / train_batch_size)
warmup_steps = int(epochs * train_data_size * 0.1 / train_batch_size)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [16]:
history = model.fit(x=train_data, #.as_numpy_iterator(),
                    epochs=2, steps_per_epoch=steps_per_epoch,
                    verbose=1)

Train for 781 steps
Epoch 1/2
781/781 [==============================] - 385s 493ms/step - loss: 0.5315 - accuracy: 0.7184
Epoch 2/2
781/781 [==============================] - 349s 446ms/step - loss: 0.3703 - accuracy: 0.8335


In [17]:
model.evaluate(test_data, verbose=1)

    781/Unknown - 162s 207ms/step - loss: 0.4478 - accuracy: 0.7921

[0.4478480930746594, 0.7921335]